資料前處理 --> 距離計算

In [14]:
from math import radians,sin,cos,tan
import numpy as np
import pandas as pd
from math import radians,sin,cos,tan
#data = pd.read_csv('./30_Training Dataset_V2/training_data.csv')
#data = pd.read_csv('./30_Public Dataset_Public Sumission Template_v2/public_dataset.csv')
data = pd.read_csv('./30_Private Dataset _Private and Publict Submission Template_v2/private_dataset.csv')

def lonlat_to_97( lon, lat):
    """
    It transforms longitude, latitude to TWD97 system.

    Parameters
    ----------
    lon : float
        longitude in degrees
    lat : float
        latitude in degrees 

    Returns
    -------
    x, y [TWD97]
    """
    
    lat = radians(lat)
    lon = radians(lon)
    
    a = 6378137.0
    b = 6356752.314245
    long0 = radians(121)
    k0 = 0.9999
    dx = 250000

    e = (1-b**2/a**2)**0.5
    e2 = e**2/(1-e**2)
    n = (a-b)/(a+b)
    nu = a/(1-(e**2)*(sin(lat)**2))**0.5
    p = lon-long0

    A = a*(1 - n + (5/4.0)*(n**2 - n**3) + (81/64.0)*(n**4  - n**5))
    B = (3*a*n/2.0)*(1 - n + (7/8.0)*(n**2 - n**3) + (55/64.0)*(n**4 - n**5))
    C = (15*a*(n**2)/16.0)*(1 - n + (3/4.0)*(n**2 - n**3))
    D = (35*a*(n**3)/48.0)*(1 - n + (11/16.0)*(n**2 - n**3))
    E = (315*a*(n**4)/51.0)*(1 - n)

    S = A*lat - B*sin(2*lat) + C*sin(4*lat) - D*sin(6*lat) + E*sin(8*lat)

    K1 = S*k0
    K2 = k0*nu*sin(2*lat)/4.0
    K3 = (k0*nu*sin(lat)*(cos(lat)**3)/24.0) * (5 - tan(lat)**2 + 9*e2*(cos(lat)**2) + 4*(e2**2)*(cos(lat)**4))

    y_97 = K1 + K2*(p**2) + K3*(p**4)

    K4 = k0*nu*cos(lat)
    K5 = (k0*nu*(cos(lat)**3)/6.0) * (1 - tan(lat)**2 + e2*(cos(lat)**2))

    x_97 = K4*p + K5*(p**3) + dx
    return x_97, y_97

university = pd.read_csv('./30_Training Dataset_V2/external_data/大學基本資料.csv',skipinitialspace = True)
station = pd.read_csv('./30_Training Dataset_V2/external_data/火車站點資料.csv',skipinitialspace = True)
hospital = pd.read_csv('./30_Training Dataset_V2/external_data/醫療機構基本資料.csv',skipinitialspace = True)

print(university.info())
for i in range(len(university)):
    university.at[i,'tw97_x'], university.at[i,'tw97_y'] = lonlat_to_97(university.loc[i,'lng'], university.loc[i,'lat'])
for i in range(len(station)):
    station.at[i,'tw97_x'], station.at[i,'tw97_y'] = lonlat_to_97(station.loc[i,'lng'], station.loc[i,'lat'])
for i in range(len(hospital)):
    hospital.at[i,'tw97_x'], hospital.at[i,'tw97_y'] = lonlat_to_97(hospital.loc[i,'lng'], hospital.loc[i,'lat'])


def near(target, currentx, currenty, city):
    distance = float('inf')
    same_city = target[target['縣市名稱'] == city]
    print('here',currentx,currenty, city)
    for j in range(len(same_city)):
        dis = np.sqrt(np.power(same_city.iloc[j]['tw97_x']-currentx,2) + np.power(same_city.iloc[j]['tw97_y']-currenty,2))
        distance = min(distance, dis)
    print(distance)
    return distance

station['郵遞區號'] = station['站點地址'].str.extract('(\d+)')
station['地址'] = station['站點地址'].str.extract('(\D+)')
station['縣市名稱'] = station['地址'].str.slice(stop=3).replace(' ',"")
hospital['縣市名稱'] = hospital['縣市鄉鎮'].str.slice(stop=3).replace(' ',"")

university['縣市名稱'] = university['縣市名稱'].str.slice(start=3)
station['縣市名稱'] = station['縣市名稱'].str.replace("臺","台")
hospital['縣市名稱'] = hospital['縣市名稱'].str.replace("臺","台")
university['縣市名稱'] = university['縣市名稱'].str.replace("臺","台")

print(station['縣市名稱'])
print(hospital['縣市名稱'])
print(university['縣市名稱'])

for i in range(len(data)):    
    data.at[i,'near_university'] = near(university, data['橫坐標'].iloc[i], data['縱坐標'].iloc[i], data['縣市'].iloc[i])
    data.at[i,'near_station'] = near(station, data['橫坐標'].iloc[i], data['縱坐標'].iloc[i], data['縣市'].iloc[i])
    data.at[i,'near_hospital'] = near(hospital, data['橫坐標'].iloc[i], data['縱坐標'].iloc[i], data['縣市'].iloc[i])

import os
os.getcwd()
#data.to_csv('new_training_data.csv')
#data.to_csv('new_public_data.csv')
data.to_csv('new_private_data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   學校代碼    791 non-null    int64  
 1   學校名稱    791 non-null    object 
 2   日間∕進修別  791 non-null    object 
 3   等級別     791 non-null    object 
 4   總計      791 non-null    int64  
 5   男生計     791 non-null    int64  
 6   女生計     791 non-null    int64  
 7   一年級男生   791 non-null    int64  
 8   一年級女生   791 non-null    int64  
 9   二年級男生   791 non-null    int64  
 10  二年級女生   791 non-null    int64  
 11  三年級男生   791 non-null    int64  
 12  三年級女生   791 non-null    int64  
 13  四年級男生   791 non-null    int64  
 14  四年級女生   791 non-null    int64  
 15  五年級男生   791 non-null    int64  
 16  五年級女生   791 non-null    int64  
 17  六年級男生   791 non-null    int64  
 18  六年級女生   791 non-null    int64  
 19  七年級男生   791 non-null    int64  
 20  七年級女生   791 non-null    int64  
 21  延修生男生   791 non-null    int64  
 22  延修

In [2]:
from math import radians,sin,cos,tan
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import xgboost

In [9]:
# 讀取訓練資料集
new_training_data = pd.read_csv("./new_training_data.csv")
train_data = pd.read_csv("./30_Training Dataset_V2/training_data.csv", encoding="utf-8")
public_dataset = pd.read_csv("./30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
private_dataset = pd.read_csv('./30_Private Dataset _Private and Publict Submission Template_v2/private_dataset.csv')
new_public_data = pd.read_csv("./new_public_data.csv")
new_private_data = pd.read_csv("./new_private_data.csv")

In [10]:
# 定義特徵和目標變數
features = ["土地面積", "屋齡", "建物面積", "車位個數", "橫坐標", "縱坐標", "near_hospital", "near_university"]
#features = ["土地面積", "屋齡", "建物面積", "車位個數", "橫坐標", "縱坐標", "near_station", "near_hospital", "near_university"]
#features = ["土地面積", "屋齡", "建物面積", "車位個數"]
target = "單價"

# 從訓練集中提取特徵和目標
#X_train = train_set[features]
#y_train = train_set[target]

#X_train = train_data[features]
#y_train = train_data[target]

X_train = new_training_data[features]
y_train = new_training_data[target]

#X_train = new_public_data[features]
#y_train = new_public_data[target]

# 從驗證集中提取特徵和目標
#X_val = val_set[features]
#y_val = val_set[target]

In [25]:
# 使用隨機森林模型進行訓練
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [11]:
# 使用xgboost.XGBRegressor()進行訓練
model = xgboost.XGBRegressor(n_estimators=78, random_state=420, learning_rate=0.2,max_depth=8,tree_method="hist",max_bin=498, 
                             num_parallel_tree=16,verbosity=0)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=498,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=78, n_jobs=None,
             num_parallel_tree=16, random_state=420, ...)

In [6]:
X_public_val = new_public_data[features]
X_private_val = new_private_data[features]
#X_private_val = private_dataset[features]
#y_val = public_dataset[target]

# 進行預測
predictions_public = model.predict(X_public_val)
predictions_private = model.predict(X_private_val)
#predictions_private = model.predict(X_private_val)

In [7]:
FinalSubmission_public = pd.DataFrame()
FinalSubmission_private = pd.DataFrame()

FinalSubmission_public['ID'] = public_dataset['ID']
FinalSubmission_public['predicted_price'] = predictions_public
FinalSubmission_public['predicted_price'] = FinalSubmission_public['predicted_price'].astype(float)

FinalSubmission_private['ID'] = private_dataset['ID']
FinalSubmission_private['predicted_price'] = predictions_private
FinalSubmission_private['predicted_price'] = FinalSubmission_private['predicted_price'].astype(float)

FinalSubmission_public = pd.concat([FinalSubmission_public, FinalSubmission_private])
FinalSubmission_public.to_csv('./concat_submission_RF.csv', index= False)

In [ ]:
############################################### 以上 ###########################################

In [9]:
FinalSubmission = pd.DataFrame()

FinalSubmission['ID'] = private_dataset['ID']
FinalSubmission['predicted_price'] = predictions_private
FinalSubmission['predicted_price'] = FinalSubmission['predicted_price'].astype(float)

FinalSubmission.to_csv('./private_submission.csv', index= False)

In [ ]:
X_val = public_dataset[features]
X_private_val = private_dataset[features]
#y_val = public_dataset[target]

# 進行預測
predictions = model.predict(X_val)
predictions_private = model.predict(X_private_val)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

new_training_data = pd.read_csv("./new_training_data.csv")
train_data = pd.read_csv("./30_Training Dataset_V2/training_data.csv", encoding="utf-8")
public_dataset = pd.read_csv("./30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
private_dataset = pd.read_csv('./30_Private Dataset _Private and Publict Submission Template_v2/private_dataset.csv')
new_public_data = pd.read_csv("./new_public_data.csv")
new_private_data = pd.read_csv("./new_private_data.csv")

In [22]:
# 切分訓練集和驗證集
train_set, val_set = train_test_split(new_training_data, test_size=0.2, random_state=42)

#features = ["土地面積", "屋齡", "建物面積", "車位個數"]
features = ["土地面積", "屋齡", "建物面積", "車位個數", "橫坐標", "縱坐標", "near_hospital", "near_university"]

target = "單價"
train_x = train_set[features]
train_y = train_set[target]
val_x = val_set[features]
val_y = val_set[target]

# 使用隨機森林模型進行訓練
"""model = RandomForestRegressor(n_estimators=500, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                              min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, 
                              oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)"""
model = RandomForestRegressor(n_estimators=300, random_state=42)
"""
# criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}
# max_features{“sqrt”, “log2”, None}, int or float, default=1.0
"""

model.fit(train_x, train_y)

# 使用xgboost.XGBRegressor()進行訓練
#model = xgboost.XGBRegressor()
#model.fit(train_x, train_y)

predictions = model.predict(val_x)

# 跟base計算MAPE
mape = np.mean(np.abs((val_y - predictions) / val_y)) * 100
print(f"Validation MAPE: {mape:.2f}%")

Validation MAPE: 9.70%


In [8]:
# 切分訓練集和驗證集
train_set, val_set = train_test_split(new_training_data, test_size=0.2, random_state=42)

#features = ["土地面積", "屋齡", "建物面積", "車位個數"]
features = ["土地面積", "屋齡", "建物面積", "車位個數", "橫坐標", "縱坐標", "near_hospital", "near_university"]

target = "單價"
train_x = train_set[features]
train_y = train_set[target]
val_x = val_set[features]
val_y = val_set[target]

# 使用隨機森林模型進行訓練
"""model = RandomForestRegressor(n_estimators=500, criterion='squared_error', max_depth=None, min_samples_split=2,
                              min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, bootstrap=True, 
                              oob_score=False, n_jobs=None, random_state=None, verbose=0)"""
model = xgboost.XGBRegressor(n_estimators=78, random_state=420, learning_rate=0.2,max_depth=8,tree_method="hist",max_bin=498, 
                             num_parallel_tree=16,verbosity=0)
"""
# criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}
# max_features{“sqrt”, “log2”, None}, int or float, default=1.0
"""

model.fit(train_x, train_y)

# 使用xgboost.XGBRegressor()進行訓練
#model = xgboost.XGBRegressor()
#model.fit(train_x, train_y)

predictions = model.predict(val_x)

# 跟base計算MAPE
mape = np.mean(np.abs((val_y - predictions) / val_y)) * 100
print(f"Validation MAPE: {mape:.2f}%")

Validation MAPE: 9.99%


In [4]:
# 讀取測試資料集
test_data_public = pd.read_csv("Public測試資料集.csv", encoding="utf-8")
test_data_private = pd.read_csv("Private測試資料集.csv", encoding="utf-8")

# 提取測試資料集的特徵
X_test_public = test_data_public[features]
X_test_private = test_data_private[features]

# 進行預測
predictions_public = model.predict(X_test_public)
predictions_private = model.predict(X_test_private)

# 建立提交檔案
submission_public = pd.DataFrame({"ID": test_data_public["ID"], "predicted_price": predictions_public})
submission_private = pd.DataFrame({"ID": test_data_private["ID"], "predicted_price": predictions_private})

# 將提交檔案儲存為.csv檔
submission_public.to_csv("submission_public.csv", index=False)
submission_private.to_csv("submission_private.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'Public測試資料集.csv'